<a href="https://colab.research.google.com/github/Sisipho-Hamlomo/Decision-tree/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import copy
init_data = {"cloud":["sun", "sun","covered", "rain", "rain", "rain", "covered", "sun","rain","sun","covered", "covered", "covered", "rain"],
        "temperature": ["hot", "hot", "hot","good", "fresh","fresh","fresh","good","fresh", "good", "good", "good", "hot", "good"],
        "humidity": ["high", "high", "high", "high", "normal", "normal", "normal", "high","normal", "normal", "normal", "high", "normal","high"],
        "wind": ["false", "true","false", "false", "false", "true", "true","false", "false", "false", "true","true", "false","true"],
         "golf": ["not play", "not play", "play", "play", "play", "not play", "play", "not play", "play", "play", "play", "play", "play", "not play"]}
#features =["cloud", "temperature", "humidity","wind"]

We first create a decision tree


In [ ]:
 #Entropy function
import numpy as np
def Entropy(target_col):
    temp = list(dict.fromkeys(target_col))             #Get unique values

    counts = [target_col.count(i) for i in temp]       #Get the number of occurances of each value from target_col

    entropy = sum([-(cnt/sum(counts))*np.log2(cnt/sum(counts)) for cnt in counts ])

    return entropy

#Auxiliary function
def getData(data, feature):
    lst = data[feature]
    t = list(dict.fromkeys(lst))
    indices = []
    for i in t:
        indices.append([a for a, x in enumerate(lst) if x == i])
    values = [[data["Golf "][i] for i in indices[k]] for k in range(len(t))]
    return lst

#Information Gain
def Gain(data, feature):

    #total Entropy
    H = Entropy(data["Golf "])

    unique = list(dict.fromkeys(data[feature]))             #Get unique values
    split = getData(data, feature)
    total_entropy = sum([(len(k)/len(data[feature]))*Entropy(k) for k in split])
    return H - total_entropy

def getRow(data, row_No):
  keys = list(data.keys())
  query = {}
  for key in keys:
    query.update({key: data[key][row_No]})

  return query


In [ ]:
def ID3(original_data, new_data, features, feature, parent_node = None):
    import pprint
    """The three stop cases are:
                **All rows in the target feautre have the same value
                **The dataset can be no longer split since the are no more features left
                **The dataset can no longer be split since there are no more rows left/ No data left
    """

    if(len(list(dict.fromkeys(new_data["Golf "])))) <= 1:
        if(len(list(dict.fromkeys(new_data["Golf "])))) == 0:
          return list(dict.fromkeys(original_data["Golf "]))[np.argmax(list(dict.fromkeys(original_data["Golf "])))]

        return list(dict.fromkeys(new_data["Golf "]))[0]

    elif(len(new_data) == 0):
        return list(dict.fromkeys(original_data["Golf "]))[np.argmax(list(dict.fromkeys(original_data["golf"])))]

    elif(len(features) == 0):
        return parent_node

    else:
        parent_node = list(dict.fromkeys(new_data[feature]))[np.argmax(list(dict.fromkeys(new_data[feature])))]

        gains = [Gain(new_data, feature) for f in features]

        bestFeatureIndex = np.argmax(gains)

        bestFeature = features[bestFeatureIndex]
        tree = {bestFeature:{}}

        features.remove(bestFeature)

        new_data1 = new_data

        for val in list(dict.fromkeys(new_data[bestFeature])):
            val = val
            z = [new_data1[bestFeature][i] == val for i in range(len(new_data1[bestFeature]))]
            indices_to_remove = [a for a, x in enumerate(z) if x == False]
            for feat in new_data1:
                for i in sorted(indices_to_remove, reverse = True):
                    lst = new_data1[feat]
                    del lst[i]

            import pandas as pd
            #print(pd.DataFrame(new_data1))
            subtree = ID3(original_data, new_data1 , features, "Golf ", parent_node)
            tree[bestFeature][val] = subtree
            #pp = pprint.PrettyPrinter(width=41, compact=True)  #Printing the tree
            #pp.pprint(tree)

        return tree

In [ ]:
def sub_sample(data, sample_size):
  dic_size = len(data)
  keys = list(data.keys())
  new_sample_arr = []
  new_sample_dict = {}
  sample_ind = np.random.randint(low=0, high=dic_size-1, size=sample_size)
  for i in sample_ind:
    new_sample_arr.append(getRow(data, i))
  for key in keys:
    new_sample_dict.update({key:[x[key] for x in new_sample_arr]})
  return new_sample_dict



In [ ]:
def random_features(data, no_features):
  keys = list(data.keys())[0:-1]
  import random
  rn = random.sample(range(0, len(keys)), no_features)
  feats = np.array(keys)[rn]
  new_dat = {}
  for f in feats:
    new_dat.update({f:data[f]})
  label_col = list(data.keys())[-1]
  new_dat.update({label_col:data[label_col]})
  return new_dat


In [ ]:
def create_forest(initial_data, num_trees):
  data = initial_data.copy()
  trees = []
  for i in range(num_trees):
    sub_data = sub_sample(data, 10)
    sample = random_features(sub_data, 3)
    feats = list(sample.keys())[0:-1]
    tree = ID3(sample, copy.deepcopy(sample), features=feats, feature="Golf ")
    trees.append(tree)
  return trees

In [ ]:
create_forest(init_data, 10)

[{'Cloud ': {'covered ': 'play', 'rain': 'play', 'sun': 'play'}},
 {'Cloud ': {'covered ': 'play', 'rain': 'play', 'sun': 'play'}},
 {'Humidity': {'high': {'Temperature': {'good': {'Wind': {'false ': 'play'}},
     'hot': 'play'}},
   'normal': 'play'}},
 {'Temperature': {'fresh': 'play',
   'good': 'play',
   'hot': {'Wind': {'false ': 'play', 'true ': 'play'}}}},
 {'Cloud ': {'covered ': 'play', 'rain': 'play', 'sun': 'play'}},
 {'Humidity': {'high': 'play',
   'normal': {'Wind': {'false ': 'play',
     'true ': {'Temperature': {'fresh': 'play', 'good': 'play'}}}}}},
 {'Temperature': {'fresh': 'play', 'good': 'play', 'hot': 'play'}},
 {'Wind': {'false ': {'Temperature': {'fresh': 'play',
     'good': {'Humidity': {'high': 'play'}},
     'hot': 'play'}},
   'true ': 'play'}},
 {'Temperature': {'fresh': 'play', 'good': 'play', 'hot': 'do not play'}},
 {'Cloud ': {'covered ': 'play', 'rain': 'play', 'sun': 'play'}}]

In [ ]:
def traverse(query, tree):
  for k in list(query.keys()):
    if k in list(tree.keys()):
      try:
        result = tree[k][query[k]]
      except:
        return 1

      res = tree[k][query[k]]

      if isinstance(result, dict):
        return traverse(query, result)
      else:
        return result

To make a prediction we will traverse all the trees in the random forest and take take the majority vote



In [ ]:
def predict(forest, query):
  from statistics import mode
  predictions = [traverse(query, tree) for tree in forest]
  return mode(predictions)


In [ ]:
import pandas as pd

Testing the Random Forest on the training data


In [ ]:
data = pd.DataFrame(init_data)
forest = create_forest(init_data, 100)
pred = []
for i in range(14):
  query = data.iloc[i].to_dict()
  query.pop("Golf ")
  pred.append(predict(forest, query))

AttributeError: ignored

In [ ]:
targets = np.array(init_data["golf"])
accuracy = ((np.array(pred) == targets)==True).sum()/14

KeyError: ignored

In [ ]:
print(accuracy)

0.42857142857142855
